<a href="https://colab.research.google.com/github/cerr/CERR/blob/octave_dev/Jupyter_Notebooks/demo_radiomic_and_dosimetric_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Example of radiomic & dosimetric feature calculation using CERR

### Install latest Octave compile



In [ ]:
%%capture
! apt-get update
! apt-get install gcc g++ gfortran make libblas-dev liblapack-dev libpcre3-dev \
libarpack2-dev libcurl4-gnutls-dev epstool libfftw3-dev transfig libfltk1.3-dev \
libfontconfig1-dev libfreetype6-dev libgl2ps-dev libglpk-dev libreadline-dev \
gnuplot-x11 libgraphicsmagick++1-dev libhdf5-serial-dev openjdk-8-jdk \
libsndfile1-dev llvm-dev lpr texinfo libgl1-mesa-dev libosmesa6-dev pstoedit \
portaudio19-dev libqhull-dev libqrupdate-dev libqscintilla2-dev \
libsuitesparse-dev texlive texlive-generic-recommended libxft-dev zlib1g-dev \
autoconf automake bison flex gperf gzip icoutils librsvg2-bin libtool perl \
rsync tar qtbase5-dev qttools5-dev qttools5-dev-tools libqscintilla2-qt5-dev \
wget git libsundials-dev gnuplot x11-apps

In [ ]:
import os, urllib.request, json

In [ ]:
# Download latest compiled octave package 
def get_octave(root_path):
  os.chdir(root_path)
  with urllib.request.urlopen("https://api.github.com/repos/cerr/octave-colab/releases/latest") as url:
      data = json.loads(url.read().decode())
  fname = data['assets'][0]['name']
  requrl = data['assets'][0]['browser_download_url']
  urllib.request.urlretrieve(requrl, fname)
  # Unzip, identify octave folder name
  !tar xf {fname}
  top_folder = !tar tf {fname} | head -1
  octave_folder = top_folder[0][:-1]
  octave_path = os.path.join(root_path,octave_folder)
  return octave_path

# Set path to Octave exectuable
octave_path = get_octave('/content')
os.environ['OCTAVE_EXECUTABLE'] = octave_path + '/bin/octave-cli'
os.environ['PATH'] = octave_path + '/bin:' + os.environ['PATH']

### Install Python-Octave bridge

In [ ]:
%%capture
! pip3 install octave_kernel
! pip3 install oct2py==5.3.0

%load_ext oct2py.ipython

### Install required Octave packages

In [ ]:
%%capture
!octave --eval "pkg install image-2.12.0.tar.gz"
!octave --eval "pkg install io-2.6.1.tar.gz"
!octave --eval "pkg install statistics-1.4.2.tar.gz"


### Download CERR


In [ ]:
%%capture
!git clone https://github.com/cerr/CERR.git
%cd /content/CERR
!git checkout octave_dev
%cd ..

### Add Octave packages and CERR to path

In [ ]:
%%octave
pkg load image
pkg load io
pkg load statistics

In [ ]:
#Add CERR-Octave to path
%%octave
cerrPath = '/content/CERR';
addpath(genpath(cerrPath))

### Read radiomics calculation settings

In [ ]:
%%octave
sampleData = '/content/CERR/Unit_Testing/data_for_cerr_tests/CERR_plans/head_neck_ex1_20may03.mat.bz2';
planC = loadPlanC(sampleData, tempdir);
planC = updatePlanFields(planC);
planC = quality_assure_planC(sampleData, planC);

sampleParam = '/content/CERR/Jupyter_Notebooks/demoParams.json';
paramS = getRadiomicsParamTemplate(sampleParam);

### Run feature calculation

In [ ]:
%%octave
indexS = planC{end};
strC = {planC{indexS.structures}.structureName};
strName = paramS.structuresC{1};
structNum = getMatchingIndex(strName,strC,'exact');
scanNum = getStructureAssociatedScan(structNum,planC);
doseNum = 1;
binWidth = 0.01;

structFieldName = ['struct_',repSpaceHyp(strName)];

In [ ]:
%%capture
%%octave

%% Compute radiomic features
radiomicsFeatS = calcGlobalRadiomicsFeatures...
(scanNum, structNum, paramS, planC);

%% Compute DVH-based features
[doseV,volV] = getDVH(structNum,doseNum,planC);
[doseBinsV,volHistV] = doseHist(doseV,volV,binWidth);
dvhFeatS.meanDose = calc_meanDose(doseBinsV, volHistV);
dvhFeatS.d10 = calc_Dx(doseBinsV, volHistV, 10, 1);
dvhFeatS.v30 = calc_Vx(doseBinsV, volHistV, 30, 0);
dvhFeatS.MOH55 = calc_MOHx(doseBinsV, volHistV, 55);

### Make computed features available in Python 


In [ ]:
from oct2py import octave

%octave_pull radiomicsFeatS dvhFeatS
print(radiomicsFeatS['shapeS'])
print(dvhFeatS)